In [5]:
import numpy as np
import h5py
import os
import csv
import time
from torch import manual_seed
from torch.utils.data import random_split
import pandas as pd

In [3]:
x = "dang"
x.split('_')

['dang']

In [28]:
os.getcwd()

'/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark'

In [6]:
train_ecg_path = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/Code15_ECG.hdf5"
train_df = 
# open hdf5
with h5py.File(train_ecg_path, "r") as f:
    
    # Pull Training ECG data
    ECG_SID_Row  = {SID:i for i,SID in enumerate(f['SID'])} # Which ECG.h5 row corresponds to which SID?
    rows_to_pull = np.array([ECG_SID_Row[SID] for SID in train_df['SID']]) # Find the rows for our dataframe
    Data['ECG_train'] = f['ECG'][np.sort(rows_to_pull)] # pull those rows in ascending order (hp5y requirement)
    train_df['ECG_Row_Num'] = rows_to_pull # sort the dataframe in ascending row-pulled order
    train_df = train_df.sort_values(by=["ECG_Row_Num"]).reset_index(drop=True)

exams_part0.hdf5
Keys: <KeysViewHDF5 ['exam_id', 'tracings']>


In [22]:
x = []
with h5py.File(os.path.join(code15_ecg_dir, "exams_part0.hdf5"), "r") as h:
    print(h['tracings'][:1, 648: -648, :].shape)
    x.append(h['exam_id'][:-1])

(1, 2800, 12)


In [25]:
np.hstack(x*2)

array([ 590673,  214626, 2936711, ..., 1444004, 2659750,  536312],
      shape=(40000,))